# Pymaceuticals Inc.
---

### Analysis

- Removing duplicates ID the total number of mice is 248. We can also see that 49% are female and 51% are male
- We can see that the correlation between mouse weight, and average tumor volume is 0.84. 
- When looking for outliers in the top 4 treatments, Infubinol was the only regimen that shows one.
- Beased in the data results it shows that Ramicane and Capomulin are the most effective treatments. 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
combined_data=pd.merge(study_results,mouse_metadata,on="Mouse ID",how="left")

# Display the data table for preview
combined_data.head()


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [ ]:
# Checking the number of mice.
mouse_metadata["Mouse ID"].count()

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
for i,x in enumerate(combined_data.duplicated(subset=['Mouse ID',"Timepoint"])):
    if x==True:
        print(combined_data.iloc[i,0:2])

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
combined_data.loc[combined_data["Mouse ID"]=="g989",:]

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
combined_data_clean=combined_data[combined_data["Mouse ID"] != 'g989']
mouse_metadata_clean=mouse_metadata[mouse_metadata["Mouse ID"] != 'g989']

In [ ]:
# Checking the number of mice in the clean DataFrame.
clean_mice_data=combined_data_clean["Mouse ID"].value_counts()
clean_number_of_mice=len(mouse_metadata_clean)
clean_number_of_mice

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
Summary_Grp=combined_data_clean.groupby(["Drug Regimen"])
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
Summary=Summary_Grp["Tumor Volume (mm3)"].mean().reset_index(name="Mean Tumor Volume")
Summary=pd.merge(Summary,Summary_Grp["Tumor Volume (mm3)"].median().reset_index(name="Median Tumor Volume"),on="Drug Regimen",how="left")
Summary=pd.merge(Summary,Summary_Grp["Tumor Volume (mm3)"].var().reset_index(name="Tumor Volume Variance"),on="Drug Regimen",how="left")
Summary=pd.merge(Summary,Summary_Grp["Tumor Volume (mm3)"].std().reset_index(name="Tumor Volume Std. Deviation"),on="Drug Regimen",how="left")
Summary=pd.merge(Summary,Summary_Grp["Tumor Volume (mm3)"].sem().reset_index(name="Tumor Volume SEM"),on="Drug Regimen",how="left")

Summary



In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
Summary2=combined_data_clean[["Drug Regimen","Tumor Volume (mm3)"]].groupby(["Drug Regimen"]).agg(['mean','median','var','std','sem'])
# Using the aggregation method, produce the same summary statistics in a single line.
Summary2

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.

count_mice_tested = combined_data_clean.groupby(["Drug Regimen"]).count()["Mouse ID"] 
plot_pandas = count_mice_tested.plot.bar(figsize=(15,10), color='b',fontsize = 12)
count_mice_tested
plt.xlabel("Drug Regimen",fontsize = 12)
plt.ylabel("Number of Mice",fontsize = 12)

plt.savefig("../Images/Pan_mice_tested_per_regimen.png", bbox_inches = "tight")
plt.tight_layout()
plt.show()

count_mice_tested

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.
mice_tested_list =(combined_data_clean.groupby(["Drug Regimen"])["Mouse ID"].count()).tolist()
mice_tested_list

x_axis = np.arange(len(count_mice_per_tret))
fig1, ax1 = plt.subplots(figsize=(15, 10))
plt.bar(x_axis, mice_tested_list, color='b', alpha=0.8, align='center')

tick_locations = [value for value in x_axis]

plt.xticks(tick_locations, ['Capomulin', 'Ceftamin', 'Infubinol', 'Ketapril', 'Naftisol', 'Placebo','Propriva', 'Ramicane', 'Stelasyn', 'Zoniferol'],  rotation='vertical')

plt.xlim(-0.75, len(x_axis)-0.25)

plt.ylim(0, max(mice_tested_list)+10)

plt.xlabel("Drug Regimen",fontsize = 12)
plt.ylabel("Number of Mice Tested",fontsize = 12)

plt.savefig("../Images/mat_mice_per_treat.png", bbox_inches = "tight")


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
combined_data_clean.groupby(["Sex"])["Mouse ID"].count().plot.pie(title="Distribution of Gender in Test",label="",autopct="%1.1f%%")


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
piePlot=combined_data_clean.groupby(["Sex"])["Mouse ID"].count().reset_index(name="Count of Mouse ID")
labs = piePlot["Sex"]
Count = piePlot["Count of Mouse ID"]
colors = ["green","blue"]
plt.pie(Count, labels=labs, colors=colors,autopct="%1.1f%%")
plt.title("Distribution of Gender in Test")

plt.show()


## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin
Top4=combined_data_clean.loc[(combined_data_clean["Drug Regimen"]=="Capomulin")|(combined_data_clean["Drug Regimen"]=="Ramicane")|(combined_data_clean["Drug Regimen"]=="Infubinol")|(combined_data_clean["Drug Regimen"]=="Ceftamin") ,:]
# Start by getting the last (greatest) timepoint for each mouse
Top4=Top4.groupby(["Drug Regimen","Mouse ID"])["Timepoint"].max().reset_index()
# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
Top4=pd.merge(Top4,combined_data_clean[["Mouse ID","Timepoint","Tumor Volume (mm3)"]],on=["Mouse ID","Timepoint"],how="left")
Top4


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
Treatments=["Capomulin","Ramicane","Infubinol","Ceftamin"]

# Create empty list to fill with tumor vol data (for plotting)
Tumors=[[] for i in range(4)]
for i in range(100):
    DR=Top4.iloc[i,:]["Drug Regimen"]
    Tum=Top4.iloc[i,:]["Tumor Volume (mm3)"]
    for j in range(4):
        if DR==Treatments[j]:
            Tumors[j].append(Tum)
            continue

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
quartiles=[[] for i in range(4)]
lowerq=[[] for i in range(4)]
upperq=[[] for i in range(4)]
iqr=[[] for i in range(4)]
lower_bound=[[] for i in range(4)]
upper_bound=[[] for i in range(4)]
potential_outliers=[[] for i in range(4)]

for Num,Treatment in enumerate(Treatments):
    quartiles[Num] = Top4.loc[Top4["Drug Regimen"]==Treatment,:]["Tumor Volume (mm3)"].quantile([.25,.5,.75])
    lowerq[Num] = quartiles[Num][0.25]
    upperq[Num] = quartiles[Num][0.75]
    iqr[Num] = upperq[Num]-lowerq[Num]

    print(f"The lower quartile of {Treatment} tumor volume is: {lowerq[Num]}")
    print(f"The upper quartile of {Treatment} tumor volume is: {upperq[Num]}")
    print(f"The interquartile range of {Treatment} tumor volume is: {iqr[Num]}")
    print(f"The the median of {Treatment} tumor volume is: {quartiles[Num][0.5]} ")

    lower_bound[Num] = lowerq[Num] - (1.5*iqr[Num])
    upper_bound[Num] = upperq[Num] + (1.5*iqr[Num])
    print(f"Values below {lower_bound[Num]} could be outliers.")
    print(f"Values above {upper_bound[Num]} could be outliers.")

    # Print Potencial outliers
    # Locate the rows which contain mice on each drug and get the tumor volumes
    # add subset 
    # Determine outliers using upper and lower bounds
    
    for Tumor in Tumors[Num]:
        if Tumor>upper_bound[Num] or Tumor < lower_bound[Num]:
            potential_outliers[Num].append(Tumor)
    if len(potential_outliers[Num])>0:
        print(f"The the potential outliers for {Treatment} are: {potential_outliers[Num]}\n")
    else:
        print(f"There are no potential outliers for {Treatment}.\n")

    

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.

fig, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4,sharey=True,figsize=(10,5))
fig.suptitle("Final Tumor Volumes by Treatment", fontsize=16, fontweight="bold")

ax1.set_ylabel('Final Tumor Volume (mm3)')
ax1.set_title(Treatments[0])
ax1.boxplot(Tumors[0])
ax1.set_xticks([])

ax2.set_title(Treatments[1])
ax2.boxplot(Tumors[1])
ax2.set_xticks([])

ax3.set_title(Treatments[2])
ax3.boxplot(Tumors[2])
ax3.set_xticks([])

ax4.set_title(Treatments[3])
ax4.boxplot(Tumors[3])
ax4.set_xticks([])

plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin
Single_Mouse_Capomulin=combined_data_clean.loc[combined_data_clean["Mouse ID"]=='l509',:]
x_axis=Single_Mouse_Capomulin["Timepoint"].count()
plt.plot(Single_Mouse_Capomulin["Timepoint"],Single_Mouse_Capomulin["Tumor Volume (mm3)"],marker='s',color="b")
plt.title("Capomulin treatment of mouse l509")
plt.xlabel("Timepoint (days)")
plt.ylabel("Tumor Volume (mm3)")
plt.ylim(40,48)
plt.show()


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
Capomulin_df = combined_data_clean.loc[combined_data_clean["Drug Regimen"] == "Capomulin",:]
Capomulin_last = Capomulin_df.groupby('Mouse ID').max()['Timepoint']
Capomulin_vol = pd.DataFrame(Capomulin_last)
Capomulin_merge = pd.merge(Capomulin_vol, combined_data_clean, on=("Mouse ID","Timepoint"),how="left")
Capomulin_merge.head()

fig1, ax1 = plt.subplots()
avg_capm_vol =Capomulin_df.groupby(['Mouse ID']).mean()

marker_size=15
plt.scatter(avg_capm_vol['Weight (g)'],avg_capm_vol['Tumor Volume (mm3)'], color="blue")
plt.title('Mouse Weight Versus Average Tumor Volume')
plt.xlabel('Weight (g)',fontsize =14)
plt.ylabel('Averag Tumor Volume (mm3)')


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model for mouse weight and average tumor volume for the Capomulin regimen
correlation = st.pearsonr(avg_capm_vol['Weight (g)'],avg_capm_vol['Tumor Volume (mm3)'])
print(f"The correlation between mouse weight and the average tumor volume is {round(correlation[0],2)}")


In [ ]:
(slope, intercept,rvalue, pvalue, stderr)= linregress(avg_capm_vol["Weight (g)"],avg_capm_vol["Tumor Volume (mm3)"])
regress_values=avg_capm_vol["Weight (g)"]* slope + intercept

plt.scatter(avg_capm_vol["Weight (g)"],avg_capm_vol["Tumor Volume (mm3)"],color='b')
plt.plot(avg_capm_vol["Weight (g)"], regress_values, color='red')
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.show()